<div class="alert alert-warning">
    
<b>Disclaimer:</b> 
    
The main objective of the <i>Jupyter</i> notebooks is to show how to use the models of the <i>QENS library</i> by
    
- building a fitting model: composition of models, convolution with a resolution function  
- setting and running the fit  
- extracting and displaying information about the results  

These steps have a minimizer-dependent syntax. That is one of the reasons why different minimizers have been used in the notebooks provided as examples.  
But, the initial guessed parameters might not be optimal, resulting in a poor fit of the reference data.

</div>

# Water Teixeira &lowast; Resolution with lmfit

## Introduction

<div class="alert alert-info">
    
This example shows how to use the <b>water_teixeira</b> model and fit the data using lmfit.
</div>
The data are two sets of water data measured at IN5 (ILL) at 5 &#x212B;.

**Reference:** J. Qvist, H. Schober and B. Halle, *J. Chem. Phys.* **134**, 144508 (2011)

### Physical units

For information about unit conversion, please refer to the jupyter notebook called `Convert_units.ipynb` in the `tools` folder.

## Import libraries

In [ ]:
import h5py
from scipy.integrate import simps
import numpy as np
import matplotlib.pyplot as plt

# the following line is to remove the warning about too many figures open simultaneously
plt. rcParams.update({'figure.max_open_warning': 0})

import lmfit
from scipy.interpolate import interp1d

%matplotlib widget

## Setting of fitting

### Import reference data

In [ ]:
path_to_data = './data/'

with h5py.File(path_to_data + 'H2O_293K_5A.hdf', 'r') as f:
    data_in = f['entry1']
    w = data_in['data1']
    x = w['X'][()]  # energy or time values
    unit_w = w['X'].attrs['long_name']
    unit_q = w['Y'].attrs['long_name']
    y = w['DATA'][()] # intensities
    e = w['errors'][()]  # errors for the intensities
    # Obtain the momentum transfer values
    q = w['Y'][()]
    data_5A = dict(q=q, x=x, y=y, e=e)

In [ ]:
# number of spectra (i.e. number of different q-values)
nb_q_values = len(data_5A['q'])

### Display units of input data
Just for information in order to determine if a conversion of units is required before using the QENSmodels

In [ ]:
print((f"The names and units of `w` (`x`axis) and `q` are: " 
       f"{unit_w[0].decode()} and {unit_q[0].decode()}, respectively."))

### Import resolution data and normalize (unit area)

In [ ]:
path_to_data = './data/'

with h5py.File(path_to_data + 'V_273K_5A.hdf', 'r') as f:
    data = f['entry1']
    w = data['data1']  
    res_5A_x = w['X'][()]
    res_5A = np.transpose(w['DATA'][()])

# Force resolution function to have unit area    
for i in range(len(data_5A['q'])):
    area = simps(res_5A[:,i], data_5A['x'])
    res_5A[:,i] /= area

### Mask data according to energy range and filter negative error

In [ ]:
# Filter according to energy-range
mask = np.intersect1d(np.where(data_5A['x']>-1.), np.where(data_5A['x']<1.))

f_5A_mask = dict()
f_5A_mask['x'] = np.asarray([data_5A['x'][mask] for i in range(nb_q_values)])
f_5A_mask['y'] = np.asarray([y[mask] for y in data_5A['y']])
f_5A_mask['e'] = np.asarray([e[mask] for e in data_5A['e']])

# Select resolution according to energy range
res_5A_x = res_5A_x[mask]
res_5A = res_5A[mask, :]

In [ ]:
# Filter according to negative error values 
# resolution
selected_indices = np.where(f_5A_mask['e'][i] > 0.0)
resol_5A_x = np.asarray([res_5A_x[selected_indices] for i in range(nb_q_values)])
resol_5A = np.asarray([res_5A[selected_indices, i][0] for i in range(nb_q_values)])                             
                              
# data
f_5A = dict()
f_5A['x'] = np.asarray([x[selected_indices] for i, x in enumerate(f_5A_mask['x'])])
f_5A['y'] = np.asarray([y[selected_indices] for i, y in enumerate(f_5A_mask['y'])])
f_5A['e'] = np.asarray([e[selected_indices] for i, e in enumerate(f_5A_mask['e'])])

In [ ]:
# plot experimental data
fig0= plt.figure()
[plt.semilogy(f_5A['x'][i], f_5A['y'][i]) for i in range(nb_q_values)]
plt.xlabel(r'Energy transfer (meV)')
plt.title('Reference data - 5 Angstrom')
plt.grid();

In [ ]:
# plot experimental data
fig1 = plt.figure()
[plt.semilogy(resol_5A_x[i], resol_5A[i]) for i in range(nb_q_values)]
plt.xlabel(r'Energy transfer (meV)') 
plt.title('Resolution function - 5 Angstrom')
plt.grid(); 

### Create function for instrument resolution data (cubic interpolation between tabulated data points)

In [ ]:
f_interp = [interp1d(resol_5A_x[i], 
                     resol_5A[i]/np.sum(resol_5A[i]), 
                     kind='cubic', 
                     bounds_error=False, 
                     fill_value='extrapolate') for i in range(nb_q_values)]

def irf_gate(w, spectrum_nb=0):
    """ Function defined from the interpolation of instrument resolution data 
    Used to define fitting model and plot """ 
    return f_interp[spectrum_nb](w)

In [ ]:
# check interpolation for first spectrum of resolution function: plot tabulated data and interpolated data
indx = 0

fig2 = plt.figure()
plt.plot(resol_5A_x[indx], 
         resol_5A[indx]/np.sum(resol_5A[indx]), 
         '.', 
         label=f"tabulated data. q={data_5A['q'][indx]:.2}")
plt.plot(f_5A['x'][indx], 
         irf_gate(f_5A['x'][indx], indx), 
         '--', 
         label=f"extrapolated data. q={data_5A['q'][indx]:.2}")
plt.legend(bbox_to_anchor=(1.0, .95))
plt.xlabel('w')
plt.title(f'Instrument resolution: tabulated data and interpolated data for spectrum {indx}')
plt.grid();

### Create fitting model

In [ ]:
import QENSmodels

In [ ]:
# Create convolution function 
# code from https://lmfit.github.io/lmfit-py/model.html

def convolve(arr, kernel):
    # simple convolution of two arrays
    npts = min(len(arr), len(kernel))
    pad  = np.ones(npts)
    tmp  = np.concatenate((pad*arr[0], arr, pad*arr[-1]))
    
    out  = np.convolve(tmp, kernel, mode='valid')
    noff = int((len(out) - npts)/2)
    return out[noff:noff+npts]

In [ ]:
model = lmfit.CompositeModel(lmfit.Model(irf_gate), lmfit.Model(QENSmodels.sqwWaterTeixeira), convolve) 

print(f'Names of parameters: {model.param_names}\nIndependent variable(s): {model.independent_vars}')

# Define boundaries for parameters to be refined
model.set_param_hint('scale', min=0, max=100)
model.set_param_hint('center', min=-0.1, max=0.1)
model.set_param_hint('D', min=0.05, max=0.25)
model.set_param_hint('resTime', min=0, max=1)
model.set_param_hint('radius', min=0.9, max=1.1)
model.set_param_hint('DR', min=0, max=1)

# Fix some of the parameters
model.set_param_hint('q', vary=False)
model.set_param_hint('spectrum_nb', vary=False)

params = model.make_params()

In [ ]:
# Plot of the fitting models without and convoluted with the resolution function
# The values of the parameters are specified below. Therefore they could be 
#different from those used in the fitting.

fig3, ax3 = plt.subplots(1, 2)
# First subplot
for i in range(nb_q_values):
    xx = f_5A['x'][i]
    ax3[0].plot(xx, 
               QENSmodels.sqwWaterTeixeira(xx, 
                                           data_5A['q'][i], 
                                           scale=1, 
                                           center=0, 
                                           D=1, 
                                           resTime=1, 
                                           radius=1, 
                                           DR=1), 
               label=f"q={data_5A['q'][i]:.2}")  

ax3[0].grid(True)
ax3[0].set(xlabel='Omega', ylabel='S(Q,w)', xlim=(-1, 1), title='No resolution') 
ax3[0].tick_params()

plt.tight_layout(rect=[0, 0, 1, 0.8])

ax3[0].legend(bbox_to_anchor=(0., 1.1, 2., 0.102), 
             loc='lower right', 
             ncol=5, 
             mode="expand", 
             borderaxespad=0., 
             fontsize=8)

# Second subplot
for i in range(nb_q_values):
    params_plot = model.make_params(nb_spectrum=i, 
                                    q=data_5A['q'][i],
                                    scale=10.,
                                    center=0.,
                                    D=0.13,
                                    resTime=0.1,
                                    radius=1.,
                                    DR=0.3)    
    xx = f_5A['x'][i]
    ax3[1].plot(xx, model.eval(params_plot, w=xx))

ax3[1].grid(True)
ax3[1].set(xlabel='w', 
          ylabel='R $\otimes$ S(Q,w)', 
          xlim=(-1,1), 
          title='Convoluted with resolution')
ax3[1].tick_params();

## Running the fit using `lmfit`

In [ ]:
ini_values = {'scale': 10., 
              'center': 0., 
              'D': 0.13, 
              'resTime': 0.1, 
              'radius': 1., 
              'DR': 0.3}

result_fit = [None,] * nb_q_values  # store fits for all spectra

for i in range(nb_q_values):
    params = model.make_params(nb_spectrum=i, 
                               q=data_5A['q'][i],
                               scale=ini_values['scale'], 
                               center=ini_values['center'], 
                               D=ini_values['D'],
                               resTime=ini_values['resTime'],
                               radius=ini_values['radius'],
                               DR=ini_values['DR'])
    
    # Q-independent parameters
    if i==0:
        D_value = params['D'].value
        resTime_value = params['resTime'].value
        radius_value = params['radius'].value
        DR_value = params['DR'].value
    else:
        params['D'].set(value=D_value)
        params['resTime'].set(value=resTime_value)
        params['radius'].set(value=radius_value)
        params['DR'].set(value=DR_value)

    result_fit[i] = model.fit(f_5A['y'][i], 
                              params,
                              w=f_5A['x'][i])

## Showing the results

using methods implemented in `lmfit`

In [ ]:
# display result
for i in range(nb_q_values):
    print(f'Result of fit {i}:\n', result_fit[i].fit_report())

Plot results using lmfit's features

In [ ]:
for i in range(nb_q_values):
    result_fit[i].plot();

Other option to plot experimental data, initial fitting model and fitted model for each spectrum using `matplotlib.pyplot`

In [ ]:
for indx in range(nb_q_values):
    fig4 = plt.figure()
    plt.plot(f_5A['x'][indx], f_5A['y'][indx], 'bo',label='exp')
    plt.plot(f_5A['x'][indx], result_fit[indx].init_fit, 'k--',label='ini')
    plt.plot(f_5A['x'][indx], result_fit[indx].best_fit, 'r-', label='fin')
    plt.title(f"q={data_5A['q'][indx]:.2}")
    plt.legend()
    plt.grid()